# 0.0.1-iw-lib-experimentation

In [1]:
from urllib.request import urlopen

import json
def get_jsonparsed_data(url):
    """
    Receive the content of ``url``, parse it as JSON and return the object.

    Parameters
    ----------
    url : str

    Returns
    -------
    dict
    """
    response = urlopen(url)
    data = response.read().decode("utf-8")
    return json.loads(data)

url = ("https://financialmodelingprep.com/api/company/profile/AAPL?datatype=json")
print(get_jsonparsed_data(url))

{'AAPL': {'Price': 223.29, 'Beta': '1.139593', 'VolAvg': '36724977', 'MktCap': '1116474338610.00', 'LastDiv': '2.92', 'Range': '142-233.47', 'Changes': 0.81, 'ChangesPerc': '(+0.36%)', 'companyName': 'Apple Inc.', 'exchange': 'Nasdaq Global Select', 'industry': 'Computer Hardware', 'website': 'http://www.apple.com', 'description': 'Apple Inc is designs, manufactures and markets mobile communication and media devices and personal computers, and sells a variety of related software, services, accessories, networking solutions and third-party digital content and applications.', 'CEO': 'Timothy D. Cook', 'sector': 'Technology', 'date_is_filed': None, 'date_bs_filed': None, 'date_cs_filed': None, 'image': 'https://financialmodelingprep.com/images-New-jpg/AAPL.jpg'}}
